Importar librerias

In [1]:
!pip install pandas
!pip install boto3
!pip install seaborn
!pip install yaml
!pip install pickle
!pip install sodapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


Importar librerias

In [2]:
import pandas as pd
import seaborn as sns
import yaml
import pickle
import boto3

from datetime import date
from sodapy import Socrata

Cargar credenciales AWS

In [3]:
#Cargar credenciales AWS
with open('/content/credencials.yaml', 'r') as file:
    config = yaml.safe_load(file)

aws_access_key_id = config['s3']['aws_access_key_id']
aws_secret_access_key = config['s3']['aws_secret_access_key']
aws_session_token = config['s3']['aws_session_token']

# funcion cargar datos historicos S3

In [11]:
session = boto3.Session(
    aws_access_key_id = config['s3']['aws_access_key_id'],
    aws_secret_access_key = config['s3']['aws_secret_access_key'],
    aws_session_token = config['s3']['aws_session_token']
)

s3 = session.client('s3')

In [44]:
bucket = "aplicaciones-cd-1-mcda24b011"
key = " ingesta/inicial/"

response = s3.list_objects_v2(Bucket=bucket, Prefix=key)

if 'Contents' in response:
    bucket_path = response['Contents'][0]['Key']
    print(f"Archivo encontrado: {bucket_path}")
else:
    print(f"No se encontraron archivos en s3://{bucket}/{key}")
    bucket_path = None  # Asignar un valor nulo si no se encuentran archivos


Archivo encontrado:  ingesta/inicial/inspecciones-historicas-2025-01-17.pkl


In [46]:
dataset = cargar_datos_s3(bucket, bucket_path)

In [47]:
len(dataset)

285708

# Transformar ingesta

In [48]:
#transformar ingesta
def transformar_ingesta(dataset):
  return pd.DataFrame.from_dict(dataset)

In [49]:
df_inspecciones = transformar_ingesta(dataset)

In [50]:
df_inspecciones.shape

(285708, 17)

In [51]:
df_inspecciones.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601,2010-01-04T00:00:00.000,Tag Removal,Pass,41.88458626715456,-87.63101044588599,"{'latitude': '41.88458626715456', 'longitude':...",NaN
1,67732,WOLCOTT'S,TROQUET,1992039,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.961605669949854,-87.67596676683779,"{'latitude': '41.961605669949854', 'longitude'...",NaN
2,52234,Cafe 608,Cafe 608,2013328,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.938006880423615,-87.6447545707008,"{'latitude': '41.938006880423615', 'longitude'...",NaN
3,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.93844282365204,-87.76831838068422,"{'latitude': '41.93844282365204', 'longitude':...",NaN
4,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,41.961605669949854,-87.67596676683779,"{'latitude': '41.961605669949854', 'longitude'...",NaN


# Faltantes

In [53]:
def faltantes(df):
  return df.isna().sum()

faltantes(df_inspecciones)

,0
inspection_id,0
dba_name,0
aka_name,2425
license_,18
facility_type,5198
risk,92
address,0
city,164
state,58
zip,43


# Eliminar faltantes latidud y longitud


In [54]:
def eliminar_faltantes_latitud_longitud(cols, df):
  for col in cols:
    df = df[df[col].notnull()]
  return df


In [56]:
inspeccion = eliminar_faltantes_latitud_longitud(['latitude', 'longitude'], df_inspecciones)
faltantes(inspeccion)

,0
inspection_id,0
dba_name,0
aka_name,2412
license_,18
facility_type,5180
risk,89
address,0
city,161
state,58
zip,40


# Imputar faltantes

In [57]:
def imputar_faltantes(cols, value, df):
  df[col].fillna(value, inplace=True)


In [59]:
#imputar columnas license, zip, state, facility_type, risk usando valores de la moda
cols = ['license_', 'zip', 'state', 'facility_type', 'risk']
for col in cols:
  imputar_faltantes(col, df_inspecciones[col].mode()[0], inspeccion)


<ipython-input-57-603d9af9cc3f>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(value, inplace=True)


In [60]:
faltantes(inspeccion)

,0
inspection_id,0
dba_name,0
aka_name,2412
license_,0
facility_type,0
risk,0
address,0
city,161
state,0
zip,0


# Transformacion de enteros

In [61]:
#inspection_id
def transformar_enteros(cols, df):
  for element in cols:
    df[element] = df[element].astype(int)
  return df

inspeccion = transformar_enteros(['inspection_id'], inspeccion)
inspeccion.dtypes

,0
inspection_id,int64
dba_name,object
aka_name,object
license_,object
facility_type,object
risk,object
address,object
city,object
state,object
zip,object


# Tranformar Flotantes

In [62]:
def transformar_flotantes(cols, df):
  for element in cols:
    df[element] = df[element].astype(float)
  return df

inspeccion = transformar_flotantes(['latitude', 'longitude'], inspeccion)
inspeccion.dtypes

,0
inspection_id,int64
dba_name,object
aka_name,object
license_,object
facility_type,object
risk,object
address,object
city,object
state,object
zip,object


# Transformar fechas

In [63]:
def transformar_fechas(cols, df):
  for element in cols:
    df[element] = df[element].astype('datetime64[ns]')
  return df

inspeccion = transformar_fechas(['inspection_date'], inspeccion)
inspeccion.dtypes

,0
inspection_id,int64
dba_name,object
aka_name,object
license_,object
facility_type,object
risk,object
address,object
city,object
state,object
zip,object


# Data Profiling String

In [65]:
def data_profiling_string(cols, df):
    data_profiling = {}
    for col in cols:
        data_profiling[col] = {'uniques': df[col].nunique(),
                               'prop_uniques': df[col].nunique()/df[col].shape[0],
                              'mode': df[col].mode()}

    return pd.DataFrame.from_dict(data_profiling)

In [67]:
data_profiling_string(['dba_name', 'aka_name', 'license_'], inspeccion)

,dba_name,aka_name,license_
uniques,33078,31478,45923
prop_uniques,0.116179,0.11056,0.161295
mode,"0 SUBWAY Name: dba_name, dtype: object","0 SUBWAY Name: aka_name, dtype: object","0 0 Name: license_, dtype: object"


# Data profiling fechas

In [69]:
def data_profiling_fechas(cols, df):
    data_profiling = {}
    for col in cols:
        data_profiling[col] = {'uniques': df[col].nunique(),
                               'prop_uniques': df[col].nunique()/df[col].shape[0],
                              'fecha_minima': df[col].min(),
                              'fecha_maxima': df[col].max(),
                              'anios_diferentes': df[col].dt.year.nunique(),
                              'dias_diferentes': df[col].max() - df[col].min()}

    return pd.DataFrame.from_dict(data_profiling)

In [70]:
data_profiling_fechas(['inspection_date'], inspeccion)

,inspection_date
uniques,3793
prop_uniques,0.013322
fecha_minima,2010-01-04 00:00:00
fecha_maxima,2025-01-16 00:00:00
anios_diferentes,16
dias_diferentes,5491 days 00:00:00


# Guardar datos limpios en s3

In [71]:
def guardar_datos_s3(bucket, bucket_path, dataset):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key'],
        aws_session_token = config['s3']['aws_session_token']
    )

    s3 = session.resource('s3')

    s3.Object(bucket, bucket_path).put(Body=dataset)

In [72]:
TODAY = date.today()

In [73]:
pickle_data = pickle.dumps(inspeccion)

In [76]:
bucket = "aplicaciones-cd-1-mcda24b011"
key = "limpieza/datos-limpios-" + str(TODAY) + ".pkl"

guardar_datos_s3(bucket, key, pickle_data)